In [1]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [2]:
install_requirements()

Installing requirements...
Requirements installed successfully.


In [ ]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

In [8]:
setup_env()

GROQ_API_KEY is set.


In [36]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union

DEFAULT_MODEL = "llama-3.3-70b-versatile"

class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))

In [39]:
from typing import List, Union, Any
from pydantic import BaseModel
from groq import Groq


class UserStory(BaseModel):
    """Represents a User Story"""
    title: str
    description: str
    acceptance_criteria: List[str]
    story_points: int

class UserStories(BaseModel):
    """Represents a collection of User Stories"""
    user_stories: List[UserStory]

class PRD(BaseModel):
    """Represents a Product Requirements Document (PRD)"""
    title: str
    overview: str
    problem_statement: str
    business_domains: List[str]
    proposed_solution: str
    target_audience: str
    features: List[str]
    constraints: List[str]
    assumptions: List[str]
    dependencies: List[str]
    user_personas: List[str]
    conclusion: str


class MiniPM:
    """An AI agent that acts as a Product Manager to help project managers create PRD and User Stories for their projects. """
    llm: Groq
    project_description: str

    def __init__(self, project_description: str):
        """Initializes the MiniPM AI agent with the project description. """
        self.project_description = project_description
        self.llm = get_groq_client()

    def get_prd(self, response_format = "markdown", augment = True) -> Union[PRD, LLMErrorResponse, Any]:
        """Gets the Product Requirements Document (PRD) for the project. """
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create a Product Requirements Document (PRD) for their projects.
                """
        prompt = f"""
                Provide a Product Requirements Document (PRD) for the project with the following details:
                Details: {self.project_description}
                """
        response = llm(
            prompt=prompt,
            response_model=PRD,
            system=system
        )

        if augment:
            response = self._augment_prd(response)

        if response_format == "json":
            return response.model_dump_json()
        elif response_format == "markdown":
            response_dict = response.model_dump()
            keys = response_dict.keys()
            markdown = f"# Product Requirements Document (PRD) for {response.title}\n"
            for key in keys:
                title = key.title().replace("_", " ").replace("Prd", "PRD")
                if key == "title":
                    title = ""
                    continue
                markdown += f"## {title}\n"
                value = response_dict[key]
                if type(value) == list:
                    value_bullets = [f"- {item}" for item in value]
                    value = "\n".join(value_bullets)
                    markdown += f"{value}\n"
                else:
                    markdown += f"{response_dict[key]}\n"
            return markdown
        elif response_format == "dto":
            return response 
        raise ValueError("Invalid response_format. Use 'json', 'markdown' or 'dto'.")
    
    def _augment_prd(self, prd: PRD) -> PRD:
        """Augments the PRD with greater details."""
        prd_json = prd.model_dump()
        print("Augmenting PRD: ", prd_json)
        keys = prd_json.keys()
        for key in keys:
            if key == "title":
                continue
            value = prd_json[key]
            prompt = f"Provide more details about the {key} for the project."
            if type(value) == str:
                prompt += f" Current value: {value}"
                expanded_value = llm(prompt, response_model=str)
                prd_json[key] = expanded_value
            elif type(value) == list:
                expanded_values = []
                for item in value:
                    prompt += f" Current value: {item}"
                    expanded_item = llm(prompt, response_model=str)
                    expanded_values.append(expanded_item)
                prd_json[key] = expanded_values
        print("Augmented PRD: ", prd_json)
        return PRD(**prd_json)


    def get_user_stories(self) -> Union[UserStories, LLMErrorResponse]:
        """Gets the User Stories for the project"""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create User Stories for their projects.
                """
        prompt = f"""
                Provide User Stories for the project with the following details:
                Details: {self.project_description}
                PRD: {self.get_prd()}
                """
        response = llm(
            prompt=prompt,
            response_model=UserStories,
            system=system
        )
        return response

In [16]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)

def save_markdown_file(markdown: str, file_path: str) -> None:
    """Saves the generated markdown file to the specified path."""
    with open(file_path, "w") as file:
        file.write(markdown)

In [38]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking investigating behavioural change in aliens."
output_file = "alien_behavioural_change_prd.md"
augment = True

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
prd = minipm.get_prd(response_format="markdown", augment=augment)

save_markdown_file(prd, output_file)

## Render the PRD as markdown
render_output(prd)


Augmenting PRD:  {'title': 'Alien Behavioural Change Tracker', 'overview': 'This web application aims to track and investigate behavioural changes in aliens, providing valuable insights into their social and cultural development.', 'problem_statement': 'The lack of a centralized platform to monitor and analyze alien behaviour hinders our understanding of their societal evolution, making it challenging to establish effective communication and cooperation.', 'business_domains': ['Astrobiology', 'Xenology', 'Artificial Intelligence'], 'proposed_solution': 'Develop a user-friendly web application that enables users to input, track, and analyze data on alien behavioural changes, utilizing machine learning algorithms to identify patterns and trends.', 'target_audience': 'Astrobiologists, Xenologists, AI Researchers, and Space Agencies', 'features': ['Data Input and Management', 'Behavioural Pattern Analysis', 'Machine Learning-based Predictions', 'Collaboration Tools', 'Data Visualization'],

# Product Requirements Document (PRD) for The project title 'Alien Behavioural Change Tracker' suggests a system designed to monitor and analyze changes in the behavior of alien species. This could be a tool for astrobiological research, allowing scientists to study and understand the patterns and habits of extraterrestrial life forms. Potential features of the tracker could include data collection mechanisms, behavioral pattern recognition algorithms, and a user interface for researchers to input observations and review findings.
## Overview
This web application, designed to track and investigate behavioural changes in aliens, provides valuable insights into their social and cultural development. It will incorporate a variety of components, including data collection modules, artificial intelligence-powered analysis tools, and a user-friendly interface for easy navigation and interpretation of results.
## Problem Statement
The absence of a unified system to track and study extraterrestrial activity significantly impedes our comprehension of their cultural and societal development. This lack of insight into their behaviour and social structures makes it extremely difficult to establish a meaningful and effective dialogue, ultimately hindering potential cooperative efforts. A centralized platform could facilitate the exchange of information, provide real-time updates, and enable the analysis of patterns and trends in alien behaviour, thereby fostering a deeper understanding of their societal evolution and potentially paving the way for mutually beneficial interactions.
## Business Domains
- Astrobiology is an interdisciplinary field that combines astronomy, biology, geology, and other sciences to study the possibility of life beyond Earth. The business domains for this project may include: Space Exploration, Biotechnology, Geospatial Analysis, and Planetary Science.
- The project's business domains currently encompass Astrobiology and Xenology. Astrobiology is the study of the origin, evolution, distribution, and future of life in the universe, while Xenology is a multidisciplinary field that focuses on the search for and study of extraterrestrial life. These domains may involve research into the potential for life on other planets, moons, and celestial bodies, the interaction between life forms and their environment, and the development of strategies for detecting and communicating with extraterrestrial life, if it exists.
- The project involves multiple business domains, including Astrobiology, which is the study of the origin, evolution, distribution, and future of life in the universe. Xenology is another key domain, focusing on the scientific study of extraterrestrial life and its potential implications. Artificial Intelligence is also a crucial aspect, as it enables the development of intelligent systems capable of analyzing complex data, making decisions, and interacting with humans. These domains are interconnected and drive innovation in areas such as space exploration, biotechnology, and machine learning.
## Proposed Solution
The proposed solution involves the development of a robust and intuitive web application that allows users to input, track, and analyze data related to alien behavioural changes. At its core, the application will utilize machine learning algorithms to identify complex patterns and trends within the data. The application will have the following key features: 1. Data Input and Validation: A form-based interface for users to input data on alien behavioural changes, along with validation checks to ensure data accuracy and consistency. 2. Data Visualization: Interactive and dynamic visualizations such as charts, graphs, and heatmaps to represent the data and facilitate easier analysis. 3. Machine Learning Integration: The application will utilize machine learning algorithms such as regression, clustering, and decision trees to analyze the data and identify patterns and trends. 4. Pattern Identification and Alerts: The application will automatically identify significant patterns and trends in the data and generate alerts for users, enabling prompt action and decision-making. 5. User Management and Authentication: A secure authentication system to ensure that only authorized users can access and manage the data and application features. 6. Data Analytics and Reporting: Advanced analytics and reporting capabilities to provide users with actionable insights and recommendations based on the data analysis.
## Target Audience
The target audience for the project includes Astrobiologists, who study the origin, evolution, distribution, and future of life in the universe. Xenologists, who focus on the search for life beyond Earth, are also key stakeholders. Additionally, AI Researchers, who develop and apply artificial intelligence to advance our understanding of space and life, are crucial to the project's success. Space Agencies, such as NASA, ESA, and others, are also primary audience members, as they are responsible for planning and executing space missions that can benefit from the project's outcomes.
## Features
- The project features Data Input for collecting user information, and Management for organizing and storing the collected data, with additional features including Data Validation for ensuring accuracy, Data Visualization for graphical representation, and Data Security for protecting user information.
- The project features Data Input and Management, which enables the collection, storage, and organization of data from various sources. Additionally, it includes Behavioural Pattern Analysis, which uses machine learning algorithms to identify trends, patterns, and correlations within the data. This allows for a deeper understanding of user behavior, prediction of future trends, and informed decision-making. Further features may include Data Visualization, Predictive Modeling, and Real-time Analytics to provide a comprehensive solution.
- The project encompasses several key features, including Data Input and Management, which enables the collection and organization of data from various sources. Behavioural Pattern Analysis is another crucial aspect, allowing for the examination and identification of patterns within the data to gain insights into user behavior. Additionally, Machine Learning-based Predictions leverage machine learning algorithms to forecast future outcomes based on historical data and trends, providing valuable predictions for informed decision-making.
- The project features Data Input and Management to collect and organize data, Behavioural Pattern Analysis to identify trends, Machine Learning-based Predictions to forecast outcomes, and Collaboration Tools to facilitate teamwork and communication.
- The project encompasses a range of comprehensive features designed to streamline data handling, analysis, and predictive modeling. Key features include: 
   1. Data Input and Management: Allows users to easily input, store, and manage data from various sources in a unified platform. 
   2. Behavioural Pattern Analysis: Enables the identification and analysis of patterns within the data to understand user or system behavior, facilitating informed decision-making. 
   3. Machine Learning-based Predictions: Employs machine learning algorithms to forecast future trends, outcomes, or behaviors based on historical data, enhancing predictive accuracy. 
   4. Collaboration Tools: Facilitates team collaboration by providing features such as real-time data sharing, task assignment, and communication tools, ensuring smooth project execution. 
   5. Data Visualization: Offers intuitive and interactive visual representations of data, making complex information more accessible and understandable for users at all levels.
## Constraints
- Data privacy and security constraints require implementing measures such as encryption, access controls, and secure data storage to protect sensitive information from unauthorized access, breaches, or other security threats.
- The project has two primary constraints: data privacy and security, as well as limited availability of alien behavioural data. Data privacy and security are crucial due to the sensitive nature of extraterrestrial research, where any information leakage could have unforeseen consequences. The limited availability of alien behavioural data hinders the project's progress, as it relies heavily on accurate and extensive data to draw meaningful conclusions about alien behaviour and preferences.
- The project is subject to several constraints, including data privacy and security concerns, limited availability of alien behavioural data, and the complexity of machine learning algorithms. Data privacy and security are crucial as the project likely involves sensitive information that must be protected from unauthorized access. Limited availability of alien behavioural data makes it challenging to develop and train accurate models, as there may not be sufficient data to work with. The complexity of machine learning algorithms requires significant computational resources and expertise to implement, train, and fine-tune.
- The project has several key constraints that need to be addressed: 
   1. **Data Privacy and Security**: Ensuring the protection of sensitive information is crucial, especially when dealing with data that may not be publicly available or is subject to strict privacy laws. 
   2. **Limited Availability of Alien Behavioural Data**: The scarcity of data related to alien behaviour significantly hampers the development and training of accurate models. 
   3. **Complexity of Machine Learning Algorithms**: Developing algorithms that can effectively learn from the limited data available, while also accounting for the unknown variables and potential biases in alien behaviour, poses a considerable challenge. 
   4. **Scalability and Performance**: The solution needs to be scalable to handle large volumes of data (when it becomes available) and perform well under various conditions, which can be particularly demanding given the unpredictability of alien behaviour and data characteristics.
## Assumptions
- Assumptions for the project include: Alien behavioural data will be available and accessible, advanced technology capable of collecting and analyzing this data will be developed, a team of expert astrobiologists and xenosociologists will be assembled to study the data, and that the aliens will not intentionally interfere with or destroy the data collection process.
- The project assumes that alien behavioural data will be available and accessible, which will be used to train machine learning algorithms. It is also assumed that these algorithms will be effective in identifying patterns within this data, allowing for insights into alien behaviour and potential communication methods. Key assumptions include: 
   1. Data Quality: The availability of high-quality, relevant, and sufficient alien behavioural data.
   2. Algorithmic Effectiveness: The capability of machine learning algorithms to accurately identify meaningful patterns within the alien behavioural data.
   3. Access to Computational Resources: Sufficient computational power and resources to process and analyze the alien behavioural data.
   4. Interdisciplinary Collaboration: Effective collaboration among experts from astrobiology, machine learning, and other relevant fields to ensure comprehensive analysis and interpretation of the data.
- For the project, it is assumed that Alien behavioural data will be available and accessible, allowing for comprehensive analysis. Additionally, it is presumed that machine learning algorithms will be effective in identifying patterns within this data. Furthermore, it is expected that users will have basic knowledge of astrobiology and xenology, enabling them to understand and interpret the findings. These assumptions provide a foundation for the project's methodology and expected outcomes.
## Dependencies
- The project relies on the availability of alien behavioural data, which is currently non-existent. Therefore, we must acknowledge that our analysis will be speculative and based on hypothetical assumptions. To overcome this limitation, we could consider alternatives such as: 
   1. Utilizing case studies from planetary exploration initiatives to identify possible patterns of extraterrestrial life.
   2. Developing machine learning models to simulate various behavioural scenarios and predict possible outcomes.
   3. Consulting with experts in astrobiology, exopaleontology, and the search for extraterrestrial intelligence (SETI) to gain insights into the potential characteristics of alien life forms.
- To further analyze the dependencies for the project, it's crucial to consider the following key points: The availability of alien behavioural data is a significant dependency, as it directly impacts our ability to understand and model extraterrestrial life forms. Additionally, advancements in machine learning and AI are essential for developing sophisticated algorithms that can process and interpret complex data sets. Other dependencies may include access to high-performance computing resources, collaboration with interdisciplinary research teams, and the development of specialized software tools for data analysis and simulation. By addressing these dependencies, we can ensure a more comprehensive understanding of the project's requirements and create a robust foundation for future research and development.
- The project dependencies include the availability of alien behavioural data, advancements in machine learning and AI, and collaboration with space agencies and research institutions. 
   To further expand on this, the project will require access to a comprehensive dataset of alien behavioural patterns, which could include social structures, communication methods, and technological capabilities. 
   Additionally, the project will rely on the latest advancements in machine learning and AI to develop predictive models that can analyze and understand alien behaviour. 
   Collaboration with space agencies and research institutions will be crucial in obtaining the necessary data and expertise to drive the project forward.
## User Personas
- Dr. Maria Rodriguez is an Astrobiologist with a Ph.D. in Astrobiology from a prestigious university. She has over 10 years of experience in the field, with research focusing on the search for life beyond Earth and the study of extreme environments. Her persona is characterized by curiosity, analytical thinking, and a passion for discovery. As a user, Dr. Rodriguez values credibility, precision, and depth of information. She is likely to engage with complex, technical content and expects accurate, up-to-date information to support her research and professional endeavors.
- The user personas for the project include two main experts: Dr. Maria Rodriguez, an astrobiologist with a background in searching for life beyond Earth, and Dr. John Lee, a xenologist specializing in the study of extraterrestrial life and its potential implications. Additional personas may include: 
   Dr. Sophia Patel, a planetary geologist interested in the geological composition of celestial bodies and its relation to potential life,
   Dr. Liam Chen, an astrophysicist focused on understanding the origins and evolution of the universe,
   and Dr. Maya Jensen, a microbiologist examining the extreme conditions under which life can thrive on Earth and potentially elsewhere.
- The user personas for the project include Dr. Maria Rodriguez, an astrobiologist searching for life beyond Earth; Dr. John Lee, a xenologist exploring the possibilities of intelligent life in the universe; and Dr. Sophia Patel, an AI researcher developing AI systems to assist in the search for life on other planets.
## Conclusion
The Alien Behavioural Change Tracker web application has the potential to revolutionize our understanding of alien societies and behaviour, enabling more effective communication and cooperation. With a user-centric design and robust machine learning capabilities, this platform will provide valuable insights for astrobiologists, xenologists, and AI researchers alike. By analyzing patterns and anomalies in alien behaviour, researchers can identify key factors that influence their social structures, decision-making processes, and interactions with other species. This knowledge can be used to inform the development of more effective strategies for intergalactic communication, cooperation, and conflict resolution, ultimately paving the way for a new era of peaceful coexistence and mutual understanding between humans and extraterrestrial life forms.


In [27]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking fitness goals and workouts."

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
user_stories = minipm.get_user_stories()

## Render the PRD as markdown
print(user_stories.model_dump_json(indent=4))

{
    "user_stories": [
        {
            "title": "User Profile Creation",
            "description": "As a user, I want to be able to create and manage my profile so that I can personalize my experience on the platform.",
            "acceptance_criteria": [
                "Users can create a new profile",
                "Users can edit their existing profile",
                "Profiles include basic user information such as name, email, and password"
            ],
            "story_points": 8
        },
        {
            "title": "Goal Setting and Tracking",
            "description": "As a user, I want to be able to set and track my fitness goals so that I can stay motivated and focused on my progress.",
            "acceptance_criteria": [
                "Users can set new fitness goals",
                "Users can view their current goals",
                "Users can track their progress towards their goals"
            ],
            "story_points": 10
        },
  